# Tunisian Finance Sentiment Analysis (XLM-RoBERTa)

## 1. Setup and Dependencies

In [ ]:
#!pip install transformers datasets torch scikit-learn pandas numpy accelerate
# Cell 2: Importing libraries 
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch

c:\Users\medam\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Data Loading & Custom Vocabulary

In [ ]:
# Cell 2: Loading custom finance vocabulary
with open('../data/reference/tunisian_finance_vocab.json', 'r', encoding='utf-8') as f:
    finance_vocab = json.load(f)

In [3]:
from sklearn.model_selection import GroupShuffleSplit


# Load dataset
data = json.load(open("../data/reference/labeled_articles.json", encoding="utf-8"))
df = pd.DataFrame(data)

# (Optional) quick sanity checks
assert {"text", "label", "group_id"}.issubset(df.columns), df.columns

# Group split to avoid leakage
texts  = df["text"].tolist()
labels = df["label"].tolist()
groups = df["group_id"].tolist()

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(texts, labels, groups))

train_df = df.iloc[train_idx].copy()
test_df  = df.iloc[test_idx].copy()

print(len(train_df), len(test_df))
print("Train distribution (raw):\n", train_df["label"].value_counts().sort_index())
print("Test distribution (raw):\n", test_df["label"].value_counts().sort_index())

# Map labels: -1→0, 0→1, 1→2 (required for num_labels=3)
label_map   = {-1: 0, 0: 1, 1: 2}
inverse_map = {0: -1, 1: 0, 2: 1}

train_df["label"] = train_df["label"].map(label_map)
test_df["label"]  = test_df["label"].map(label_map)

# Fail fast if anything wasn't mapped
if train_df["label"].isna().any() or test_df["label"].isna().any():
    bad_train = df.iloc[train_idx][df.iloc[train_idx]["label"].map(label_map).isna()]["label"].unique()
    bad_test  = df.iloc[test_idx][df.iloc[test_idx]["label"].map(label_map).isna()]["label"].unique()
    raise ValueError(f"Found unmapped labels. Bad train labels: {bad_train}, bad test labels: {bad_test}")

train_df["label"] = train_df["label"].astype(int)
test_df["label"]  = test_df["label"].astype(int)

print("Train distribution (mapped):\n", train_df["label"].value_counts().sort_index())
print("Test distribution (mapped):\n", test_df["label"].value_counts().sort_index())
print("Unique labels train/test:", sorted(train_df["label"].unique()), sorted(test_df["label"].unique()))


200 50
Train distribution (raw):
 label
-1    59
 0    71
 1    70
Name: count, dtype: int64
Test distribution (raw):
 label
-1    11
 0    19
 1    20
Name: count, dtype: int64
Train distribution (mapped):
 label
0    59
1    71
2    70
Name: count, dtype: int64
Test distribution (mapped):
 label
0    11
1    19
2    20
Name: count, dtype: int64
Unique labels train/test: [np.int64(0), np.int64(1), np.int64(2)] [np.int64(0), np.int64(1), np.int64(2)]


In [4]:
# Cell 5: Creating Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label']])


## 3. Tokenization 

In [5]:
# Cell 6: Initialize XLM-RoBERTa tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add custom finance tokens (important!)
new_tokens = (
    finance_vocab['entities'] + 
    finance_vocab['financial_terms_ar'] + 
    finance_vocab['financial_terms_fr']
)
tokenizer.add_tokens(new_tokens)

349

In [6]:
# Cell 7: Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['text'], 
        padding='max_length', 
        truncation=True,
        max_length=256  
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 50/50 [00:00<00:00, 1551.30 examples/s]


In [7]:
# Cell 8: Load XLM-RoBERTa model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label=inverse_map,
    label2id=label_map
)

# Resize embeddings for new tokens
model.resize_token_embeddings(len(tokenizer))

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(250331, 768, padding_idx=1)

## 4. Training Configuration

In [8]:
# Cell 9 : Training arguments (optimized for financial sentiment)
training_args = TrainingArguments(
    output_dir="./checkpoints/xlm-roberta-tunisian-finance",
    num_train_epochs=10,  
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_ratio=0.1,              #warmup_steps=50,
    weight_decay=0.01,
    learning_rate=1e-5,  #change from 2e-5 to 1e-5 for better convergence on small dataset
    logging_dir='./checkpoints/logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    seed=42
)


c:\Users\medam\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
# Cell 11: Metrics computation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    return {
        "accuracy": accuracy_score(labels, predictions),
        "report": classification_report(
            labels, 
            predictions, 
            target_names=['Negative', 'Neutral', 'Positive'],
            output_dict=True
        )
    }

In [10]:
# Check exact duplicates between train and test
overlap = set(train_df['text']) & set(test_df['text'])
print(f"Overlapping samples: {len(overlap)}")


Overlapping samples: 0


In [11]:
# Cell 12: Initialize Trainer
from transformers import EarlyStoppingCallback


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


C:\Users\medam\AppData\Local\Temp\ipykernel_15512\3938051043.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## 5. Training & Evaluation

In [12]:
# Cell 13: Train!
print("Starting fine-tuning...")
trainer.train()

Starting fine-tuning...


  0%|          | 0/250 [00:00<?, ?it/s]c:\Users\medam\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
  4%|▍         | 10/250 [02:09<55:13, 13.81s/it]

{'loss': 1.0899, 'grad_norm': 4.572332382202148, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.4}


  8%|▊         | 20/250 [04:24<53:36, 13.99s/it]

{'loss': 1.0906, 'grad_norm': 2.6576404571533203, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.8}


...

                                                 
 80%|████████  | 200/250 [47:34<09:36, 11.53s/it]

{'eval_loss': 0.7864121198654175, 'eval_accuracy': 0.7, 'eval_report': {'Negative': {'precision': 0.45, 'recall': 0.8181818181818182, 'f1-score': 0.5806451612903226, 'support': 11.0}, 'Neutral': {'precision': 1.0, 'recall': 0.7894736842105263, 'f1-score': 0.8823529411764706, 'support': 19.0}, 'Positive': {'precision': 0.7333333333333333, 'recall': 0.55, 'f1-score': 0.6285714285714286, 'support': 20.0}, 'accuracy': 0.7, 'macro avg': {'precision': 0.7277777777777777, 'recall': 0.7192185007974482, 'f1-score': 0.6971898436794072, 'support': 50.0}, 'weighted avg': {'precision': 0.7723333333333333, 'recall': 0.7, 'f1-score': 0.7144646245595012, 'support': 50.0}}, 'eval_runtime': 13.1813, 'eval_samples_per_second': 3.793, 'eval_steps_per_second': 0.531, 'epoch': 8.0}


 80%|████████  | 200/250 [47:47<11:56, 14.34s/it]

{'train_runtime': 2867.2526, 'train_samples_per_second': 0.698, 'train_steps_per_second': 0.087, 'train_loss': 0.8130919170379639, 'epoch': 8.0}


TrainOutput(global_step=200, training_loss=0.8130919170379639, metrics={'train_runtime': 2867.2526, 'train_samples_per_second': 0.698, 'train_steps_per_second': 0.087, 'total_flos': 210490734182400.0, 'train_loss': 0.8130919170379639, 'epoch': 8.0})

In [13]:
# Cell 14: Evaluate
results = trainer.evaluate()
print(f"\n{'='*60}")
print(f"Test Accuracy: {results['eval_accuracy']:.2%}")
print(f"{'='*60}")

c:\Users\medam\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


100%|██████████| 7/7 [00:11<00:00,  1.70s/it]


Test Accuracy: 72.00%


In [2]:
import os, glob, json

out_dir = "./checkpoints/xlm-roberta-tunisian-finance"

print("Output dir exists:", os.path.exists(out_dir))
print("Checkpoint dirs:", glob.glob(os.path.join(out_dir, "checkpoint-*")))


Output dir exists: True
Checkpoint dirs: ['./checkpoints/xlm-roberta-tunisian-finance\\checkpoint-100', './checkpoints/xlm-roberta-tunisian-finance\\checkpoint-150']


In [6]:
# Cell 15: Save model
model.save_pretrained("../src/models/xlm-roberta-tunisian-finance-final")
tokenizer.save_pretrained("../src/models/xlm-roberta-tunisian-finance-final")
print("✓ Model saved!")

✓ Model saved!
